In [1]:
import numpy as np
#from scipy.misc import imread, imsave, imresize
from skimage.io import imread, imsave
from skimage.transform import resize as imresize
from scipy.spatial.distance import cosine
import os
import sys
import glob
import time
import pandas as pd
#sys.path.append('./utils/')
from utils.rgb_ind_convertor import *

In [29]:
baseDirectory = '/d2/studies/TF2DeepFloorplan/'
benchmark_path='/d2/studies/TF2DeepFloorplan/dataset/r3d_train.txt'
result_dir='/d2/studies/TF2DeepFloorplan/outJan19_TFR5_lr5e-4_ss25/450/' #change this to outdir of training
num_of_classes=11

In [25]:
def evaluateBounds_cosine(benchmark_path, result_dir, suffix=None, im_resize=True, gt_resize=True, train=False):
    """

    Parameters
    ----------
    benchmark_path : string
        Path to .txt file containing dataset paths.
    result_dir : string
        Directory containing result images to evaluate.
    suffix : string, optional
        Suffix to append to file name
    im_resize : boolean, optional
        Whether to resize result images to 512x512. The default is True.
    gt_resize : boolean, optional
        Whether to resize ground truth images to 512x512. The default is True.
    train : boolean, optional
        Whether to evaluate training data or test data. The default is False.

    Returns
    -------
    None.

    """
    gt_paths = open(benchmark_path, 'r').read().splitlines()
    d_paths = [p.split('\t')[2] for p in gt_paths] # 1 denote wall, 2 denote door, 3 denote room
    r_paths = [p.split('\t')[3] for p in gt_paths] # 1 denote wall, 2 denote door, 3 denote room
    cw_paths = [p.split('\t')[-1] for p in gt_paths] # 1 denote wall, 2 denote door, 3 denote room, last one denote close wall
    im_names = [p.split('/')[-1].split('.')[0] for p in gt_paths]
    im_paths = [os.path.join(result_dir, p.split('/')[-1].split('.')[0]).replace('_close', '_doors_windows') + '.png' for p in d_paths]
    data_dir = os.path.dirname(benchmark_path)
    sims=[]
    names = []
    for i in range(len(im_paths)):
        try:
            res_im  = imread(im_paths[i], pilmode='L')
            name = os.path.basename(im_paths[i]).split('_')[0]
            if train:
                gt_im = imread(os.path.join(data_dir, 'newyork/train/' + name + '_close.png'), pilmode='L')
            elif not train:
                gt_im = imread(os.path.join(data_dir, 'newyork/test/' + name + '_close.png'), pilmode='L')
                
            if im_resize:
                res_im = imresize(res_im, (512,512,3), mode='constant', cval=0, preserve_range=True)
            if gt_resize:
                gt_im = imresize(gt_im, (512,512,3), mode='constant', cval=0, preserve_range=True)
            res_im_1d = res_im.flatten()
            gt_im_1d = gt_im.flatten()
            res_im_1d = res_im_1d + 1e-6
            gt_im_1d = gt_im_1d + 1e-6
            cos = cosine(res_im_1d, gt_im_1d)
            sim = 1-cos
            print("Image " + str(name) + " similarity " + str(sim))
            sims.append(sim)
            names.append(name)
        except FileNotFoundError:
            pass
    df = pd.DataFrame([names, sims]).T
    df.columns=['image', 'similarity']
    df.sort_values(by='similarity', inplace=True, ascending=False)
    df.reset_index(drop=True, inplace=True)
    if suffix:
        df.to_csv('Cosine_EntranceSimilarity_Results_' + suffix + '.csv')
    elif not suffix:
        df.to_csv('Cosine_EntranceSimilarity_Results.csv')
    aveSim = np.mean(sims)
    print("Average cosine similarity " + str(aveSim))
    return df

In [9]:
def evaluateRooms_cosine(benchmark_path, result_dir, num_of_classes=11, suffix=None, im_resize=True, gt_resize=True, train=False):
    """

    Parameters
    ----------
    benchmark_path : string
        Path to .txt file containing dataset paths.
    result_dir : string
        Directory containing result images to evaluate.
    num_of_classes : int, optional
        Number of room classes. The default is 11.
    suffix : string, optional
        Suffix to append to file name.
    im_resize : boolean, optional
        Whether to resize result images to 512x512. The default is True.
    gt_resize : boolean, optional
        Whether to resize ground truth images to 512x512. The default is True.
    train : boolean, optional
        Whether to evaluate training data or test data. The default is False.

    Returns
    -------
    None.

    """
    gt_paths = open(benchmark_path, 'r').read().splitlines()
    r_paths = [p.split('\t')[3] for p in gt_paths] # 1 denote wall, 2 denote door, 3 denote room
    im_names = [p.split('/')[-1].split('.')[0] for p in gt_paths]
    im_paths = [os.path.join(result_dir, p.split('/')[-1].split('.')[0] + '_pred.png') for p in r_paths]
    data_dir = os.path.dirname(benchmark_path)
    sims=[]
    names = []
    for i in range(len(im_paths)):
        try:
            res_im  = imread(im_paths[i], pilmode='RGB')
            name = os.path.basename(im_paths[i]).split('_')[0]
            if train:
                gt_im = imread(os.path.join(data_dir, 'newyork/train/' + name + '_multi.png'), pilmode='RGB')
                gt_im_ind = rgb2ind(gt_im, color_map=floorplan_fuse_map) 
            elif not train:
                gt_im = imread(os.path.join(data_dir, 'newyork/test/' + name + '_multi.png'))
                gt_im_ind = rgb2ind(im, color_map=floorplan_fuse_map)
                
            if im_resize:
                res_im = imresize(res_im, (512,512,3), mode='constant', cval=0, preserve_range=True)
            if gt_resize:
                gt_im_ind = imresize(gt_im_ind, (512,512,3), mode='constant', cval=0, preserve_range=True)
            res_im_1d = res_im.flatten()
            gt_im_1d = gt_im_ind.flatten()
            res_im_1d = res_im_1d + 1e-6
            gt_im_1d = gt_im_1d + 1e-6
            cos = cosine(res_im_1d, gt_im_1d)
            sim = 1-cos
            print("Image " + str(name) + " similarity " + str(sim))
            sims.append(sim)
            names.append(name)
        except FileNotFoundError:
            pass
    df = pd.DataFrame([names, sims]).T
    df.columns=['image', 'similarity']
    df.sort_values(by='similarity', inplace=True, ascending=False)
    df.reset_index(drop=True, inplace=True)
    if suffix:  
        df.to_csv('Cosine_Similarity_Results_' + suffix + '.csv')
    elif not suffix:
            df.to_csv('Cosine_RoomSimilarity_Results.csv')

    aveSim = np.mean(sims)
    print("Average cosine similarity " + str(aveSim))
    return df


In [20]:
evaluateRooms_cosine(benchmark_path, result_dir, num_of_classes=11, im_resize=True, gt_resize=True, train=True)

Image 10 similarity 0.4977814654142483
Image 28025487 similarity 0.3066534668374228
Image 28906422 similarity 0.34213068187279894
Image 2 similarity 0.44144078276635434
Image 30044076 similarity 0.29213759541953643
Image 30049107 similarity 0.3267587788167189
Image 30615117 similarity 0.385189842548723
Image 30939153 similarity 0.35139552644796423
Image 30957516 similarity 0.2797476322639554
Image 31036152 similarity 0.30199046259741746
Image 31234182 similarity 0.4444767108133235
Image 31272420 similarity 0.3232989054439048
Image 31318404 similarity 0.36933293329423056
Image 31418847 similarity 0.40666366242377094
Image 31431717 similarity 0.37943788985765936
Image 31450071 similarity 0.34436121979139167
Image 31483593 similarity 0.3665285911623666
Image 31491612 similarity 0.2922317533167287
Image 31566489 similarity 0.2756476348653836
Image 31567842 similarity 0.36607823873695056
Image 31573533 similarity 0.32518011995659757
Image 31677402 similarity 0.31795626149167733
Image 316831

,image,similarity
0,10,0.497781
1,46,0.493150
2,45743284,0.468494
3,47360870,0.460073
4,31234182,0.444477
...,...,...
174,45725026,0.266650
175,31850409,0.260751
176,31850325,0.254666
177,45765070,0.236102


In [30]:
evaluateBounds_cosine(benchmark_path, result_dir, suffix='Jan19_Epoch0', im_resize=True, gt_resize=True, train=True)

Image 10 similarity 0.9808341148594383
Image 28025487 similarity 0.9922990317684417
Image 28906422 similarity 0.9717688312063891
Image 2 similarity 0.9828153304243064
Image 30044076 similarity 0.9890715606122288
Image 30049107 similarity 0.9941569661191979
Image 30615117 similarity 0.9785925212676458
Image 30939153 similarity 0.9845735597797072
Image 30957516 similarity 0.9871493028057534
Image 31036152 similarity 0.9852286366263803
Image 31234182 similarity 0.9867591463127585
Image 31272420 similarity 0.9932413276784365
Image 31318404 similarity 0.9883582081710129
Image 31418847 similarity 0.9824853900601296
Image 31431717 similarity 0.9827637487847768
Image 31450071 similarity 0.9812295767708327
Image 31483593 similarity 0.9823077472285265
Image 31491612 similarity 0.988426047191923
Image 31566489 similarity 0.9837971042699866
Image 31567842 similarity 0.9878867929754009
Image 31573533 similarity 0.9796911260683349
Image 31677402 similarity 0.9874173390269874
Image 31683135 similarit

,image,similarity
0,45764671,0.995972
1,45780376,0.994242
2,30049107,0.994157
3,45777694,0.993431
4,31272420,0.993241
...,...,...
174,45775894,0.972169
175,31852926,0.971924
176,28906422,0.971769
177,31890228,0.971337


'/d2/studies/TF2DeepFloorplan/outJan19_TFR5_lr5e-4_ss25/0/'